In [3]:
import ipyparallel as ipp
c = ipp.Client()
dview = c[:]

In [7]:
%%px --local
import sys

import sigproc
from sigproc import dtw
import numpy as np
import numpy.ma as ma
import time

In [4]:
def mul(a, b):
    return a * b

In [8]:
N = 10**7
tm = time.time()
list(map(mul, np.arange(N), np.arange(N, N+N)))
tm = time.time() - tm
print(tm)

3.850149154663086


In [9]:
res = dview.map_async(mul, np.arange(N), np.arange(N, N+N))

In [10]:
tm = time.time()
res.get()
tm = time.time() - tm
print(tm)

KeyboardInterrupt: 

In [14]:
%%px --local
import sys

import sigproc
from sigproc import dtw
import numpy as np
import numpy.ma as ma
import time 

In [26]:
%%px --local
data = np.random.rand(40, 100)
data = ma.masked_array(data)

In [22]:
v0, v1, v2, v3 = c[0], c[1], c[2], c[3]

In [19]:
s1 = v0.apply_async(sigproc.m_distance_features, ipp.Reference(data))
s2 = v1.apply_async(sigproc.m_distance_features, data)
s3 = v2.apply_async(sigproc.m_distance_features, data)
s4 = v3.apply_async(sigproc.m_distance_features, data)

TypeError: names must be strs, not 0

In [129]:
serial_result = map(lambda x:x**10, range(32))

In [130]:
parallel_result = dview.map_sync(lambda x: x**10, range(32))

list(serial_result)==parallel_result

True

In [128]:
np.array(list(parallel_result)) - np.array(list(serial_result))

array([0, 0, 0, ..., 0, 0, 0])

In [83]:
dview['data'][0].shape

(10, 100)

In [23]:
dview.scatter('data', data)

<AsyncResult: scatter>

In [25]:
ipp.Reference('data')

array(<Reference: 'data'>, dtype=object)

In [19]:
f = ipp.interactive(sigproc.m_distance_features)
f = ipp.interactive(sigproc.dtw)
#dview['dtw'] = sigproc.dtw

<AsyncResult: finished>

In [28]:
%timeit dists = dview.apply_sync(sigproc.m_distance_features, ipp.Reference('data'), np.array([0, 0, 0]))

1 loops, best of 3: 4.74 s per loop


In [29]:
%timeit dists = dview.apply_async(sigproc.m_distance_features, ipp.Reference('data'), np.array([0, 0, 0]))

100 loops, best of 3: 5.13 ms per loop


In [67]:
tm = time.time()
dists1 = dview.apply_sync(sigproc.m_distance_features, ipp.Reference('data'), np.array([0, 0, 0]))
tm = time.time() - tm
print(tm)

7.166122198104858


In [68]:
tm = time.time()
dists2 = dview.apply_async(sigproc.m_distance_features, ipp.Reference('data'), np.array([0, 0, 0]))
tm = time.time() - tm
print(tm)

0.005227088928222656


In [69]:
tm = time.time()
dists2 = dists2.get()
tm = time.time() - tm
print(tm)

6.981907844543457


In [75]:
np.all(np.equal(dists1, dists2))

True

In [76]:
dists3 = np.empty((4, 10, 10))
tm = time.time()
for i in range(4):
    dists3[i] = sigproc.m_distance_features(data[10*i:10*(i+1)], np.array([0, 0, 0]))
tm = time.time() - tm

In [77]:
print(tm)

11.166589260101318


In [79]:
np.all(np.equal(dists2, dists3))

True